In [1]:
#Importar librerias
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import call_udf
from config import connection_parameters
from ingesta import ingesta_setup,solo_archivos
import pandas as pd
import streamlit as st
import json
import csv
import gzip
import os

In [2]:
#Definir sesión y objetos Snowflake
sesion = Session.builder.configs(connection_parameters).create()
print("Conectado")

2022-12-15 19:44:50.960 INFO    snowflake.connector.connection: Snowflake Connector for Python Version: 2.7.12, Python Version: 3.8.13, Platform: macOS-10.16-x86_64-i386-64bit
2022-12-15 19:44:50.961 INFO    snowflake.connector.connection: This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2022-12-15 19:44:50.962 INFO    snowflake.connector.connection: Setting use_openssl_only mode to False
2022-12-15 19:44:52.217 INFO    snowflake.snowpark.session: Snowpark Session information: 
"version" : 0.12.0,
"python.version" : 3.8.13,
"python.connector.version" : 2.7.12,
"python.connector.session.id" : 593023319285782,
"os.name" : Darwin



Conectado


In [3]:
#Snowflake Objetos
sesion.use_role("devrole")
sesion.use_database("INEGI")
sesion.use_warehouse("PYTHON_WH")
sesion.use_schema("PUBLIC")
print(sesion.sql("select current_warehouse(), current_database(), current_schema()").collect()) 

2022-12-15 19:44:56.913 INFO    snowflake.connector.cursor: query: [use role devrole]
2022-12-15 19:44:57.139 INFO    snowflake.connector.cursor: query execution done
2022-12-15 19:44:57.140 INFO    snowflake.connector.cursor: query: [use database INEGI]
2022-12-15 19:44:57.301 INFO    snowflake.connector.cursor: query execution done
2022-12-15 19:44:57.302 INFO    snowflake.connector.cursor: query: [use warehouse PYTHON_WH]
2022-12-15 19:44:57.423 INFO    snowflake.connector.cursor: query execution done
2022-12-15 19:44:57.424 INFO    snowflake.connector.cursor: query: [use schema PUBLIC]
2022-12-15 19:44:57.554 INFO    snowflake.connector.cursor: query execution done
2022-12-15 19:44:57.558 INFO    snowflake.connector.cursor: query: [select current_warehouse(), current_database(), current_schema()]
2022-12-15 19:44:57.711 INFO    snowflake.connector.cursor: query execution done


[Row(CURRENT_WAREHOUSE()='PYTHON_WH', CURRENT_DATABASE()='INEGI', CURRENT_SCHEMA()='PUBLIC')]


In [4]:

#Crear internal Stage
env = ingesta_setup()
sesion.sql('CREATE STAGE IF NOT EXISTS '+ env['snowstage']).collect()
#Upload archivos
archivos_dir = os.path.join(os.getcwd(),'JSON')
archivos = solo_archivos(archivos_dir)

for file in archivos:
    put_result = sesion.file.put('file:///' + file , '@' + env['snowstage'])
    print(put_result[0].status)
    #Copy_into


2022-12-15 19:45:03.051 INFO    snowflake.connector.cursor: query: [CREATE STAGE IF NOT EXISTS inegi]
2022-12-15 19:45:03.259 INFO    snowflake.connector.cursor: query execution done
2022-12-15 19:45:03.262 INFO    snowflake.connector.cursor: query: [PUT 'file:////Users/csuarez/Documents/DEV/VOHL/NEGI/JSON/inegi3.json.gz' '@inegi...]
2022-12-15 19:45:03.524 INFO    snowflake.connector.cursor: query execution done
2022-12-15 19:45:07.518 INFO    snowflake.connector.cursor: query: [PUT 'file:////Users/csuarez/Documents/DEV/VOHL/NEGI/JSON/inegi2.json.gz' '@inegi...]


UPLOADED


2022-12-15 19:45:07.747 INFO    snowflake.connector.cursor: query execution done
2022-12-15 19:45:09.359 INFO    snowflake.connector.cursor: query: [PUT 'file:////Users/csuarez/Documents/DEV/VOHL/NEGI/JSON/inegi1.json.gz' '@inegi...]


UPLOADED


2022-12-15 19:45:09.661 INFO    snowflake.connector.cursor: query execution done
2022-12-15 19:45:10.982 INFO    snowflake.connector.cursor: query: [PUT 'file:////Users/csuarez/Documents/DEV/VOHL/NEGI/JSON/inegi8.json.gz' '@inegi...]


...

UPLOADED


2022-12-15 19:45:14.795 INFO    snowflake.connector.cursor: query: [PUT 'file:////Users/csuarez/Documents/DEV/VOHL/NEGI/JSON/inegi6.json.gz' '@inegi...]


UPLOADED


2022-12-15 19:45:15.005 INFO    snowflake.connector.cursor: query execution done


UPLOADED


In [5]:
sesion.close()
print("Sesión terminada")

2022-12-15 19:46:51.406 INFO    snowflake.snowpark.session: Closing session: 593023319285782
2022-12-15 19:46:51.409 INFO    snowflake.snowpark.session: Canceling all running queries
2022-12-15 19:46:51.409 INFO    snowflake.connector.cursor: query: [select system$cancel_all_queries(593023319285782)]
2022-12-15 19:46:51.558 INFO    snowflake.connector.cursor: query execution done
2022-12-15 19:46:51.563 INFO    snowflake.connector.connection: closed
2022-12-15 19:46:51.657 INFO    snowflake.connector.connection: No async queries seem to be running, deleting session
2022-12-15 19:46:51.797 INFO    snowflake.snowpark.session: Closed session: 593023319285782


Sesión terminada
